In [1]:
#bigram 써서 해보기
from nltk.tokenize import regexp_tokenize

In [3]:
import re
from os import listdir

def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(''.join(s[i:i+n]))
    return rst

def fileids(path, ext='txt'):
    files = list(filter(lambda f:re.search(f'{ext}$', f),
                       listdir(path)))
    return list(map(lambda f:f'{path}/{f}', files))

In [4]:
import re
from os import listdir

p1 = re.compile(r'\s+')

D = list()
V = list()
TDM  = dict()


for file in fileids('naver'):
    i = len(D)
    D.append(file)
    with open(file, 'r', encoding='utf8') as fp:
        for t in regexp_tokenize(p1.sub(' ', fp.read()), r'\b\w+\b'):
            for g in ngram(t):
                if g not in V:
                    V.append(g)
                j = V.index(g)

                if j not in TDM.keys():
                    TDM[j] = list()

                if i not in TDM[j]:
                    #이 안에는 숫자들로만 구성되어있을거다
                    TDM[j].append(i)

In [5]:
len(TDM), len(V), len(D), len(TDM[0])

(27694, 27694, 387, 21)

In [6]:
sum([len(v) for k,v in TDM.items()])/len(V) # =>|L|

5.984220408752798

In [7]:
i2w = lambda i:V[i]
w2i = lambda w:V.index(w)

i2d = lambda i:D[i]
d2i = lambda d:D.index(d)

In [15]:
#사용자에게서 쿼리를 받아보기
query = '예상보다 큰 격차의 참패로 여권이 대혼란에 빠지자 윤석열 대통령은 김행 여성가족부 장관 후보자 지명 철회 카드로 수습에 나섰다.'

result = list()

for t in regexp_tokenize(query, r'\b\w+\b'):
    #ngram 썼으니까 for문 한번더 돌리기
    for g in ngram(t): #ngram = > tokenizing.. 
        if g in V:
            j = w2i(t)
            result.append(TDM[j])

In [11]:
#OR
# candidates = list()
# for r in result:
#     candidates.extend(r)
# list(set(candidates))

#AND
candidates = list()
for r in result:
    candidates = list(set(candidates).intersection(r))
candidates
#결과 없는게 맞다.

[]

## 3번방법. 리스트로 표현하기

In [16]:
#위치값만 정해줄거임.
# TDM = {
#     {Term1:Posting 몇 번째 위치}
# }
# Posting = [
#             0:(doc1, 
# ]


p1 = re.compile(r'\s+')

D = list()
V = list()
TDM  = dict()
Posting = list()


for file in fileids('naver'):
    i = len(D)
    D.append(file)
    with open(file, 'r', encoding='utf8') as fp:
        for t in regexp_tokenize(p1.sub(' ', fp.read()), r'\b\w+\b'):
            for g in ngram(t):
                if g not in V:
                    V.append(g)
                j = V.index(g)

                if j not in TDM.keys():
                    #현재 위치를 기록
                    pos = len(Posting)
                    Posting.append((i,1,-1))
                    #위치를 돌려줄거
                    TDM[j] = pos
                else:
                    pos = len(Posting)
                    Posting.append((i,1,TDM[j]))
                    TDM[j] = pos


In [17]:
TDM

{0: 64377,
 1: 179542,
 2: 1114,
 3: 265899,
 4: 270144,
 5: 273053,
 6: 254041,
 7: 211269,
 8: 259787,
 9: 250857,
 10: 270023,
 11: 210420,
 12: 167,
 13: 237548,
 14: 257032,
 15: 241396,
 16: 216889,
 17: 272511,
 18: 258877,
 19: 264399,
 20: 269003,
 21: 254189,
 22: 253545,
 23: 270736,
 24: 270298,
 25: 220199,
 26: 258037,
 27: 269681,
 28: 271133,
 29: 206905,
 30: 206907,
 31: 231355,
 32: 258199,
 33: 1297,
 34: 255293,
 35: 269068,
 36: 157752,
 37: 157753,
 38: 268308,
 39: 272075,
 40: 202799,
 41: 245211,
 42: 267317,
 43: 273202,
 44: 231950,
 45: 522,
 46: 523,
 47: 273127,
 48: 63097,
 49: 245721,
 50: 265873,
 51: 265874,
 52: 262442,
 53: 217886,
 54: 273593,
 55: 272656,
 56: 65,
 57: 263466,
 58: 183,
 59: 270767,
 60: 273247,
 61: 272378,
 62: 272379,
 63: 245355,
 64: 267488,
 65: 2733,
 66: 257848,
 67: 273548,
 68: 267236,
 69: 267386,
 70: 258115,
 71: 273427,
 72: 267839,
 73: 86,
 74: 87,
 75: 264241,
 76: 273070,
 77: 243079,
 78: 267456,
 79: 252749,
 8

In [19]:
#사용자에게서 쿼리를 받아보기
query = '예상보다 큰 격차의 참패로 여권이 대혼란에 빠지자 윤석열 대통령은 김행 여성가족부 장관 후보자 지명 철회 카드로 수습에 나섰다.'

result = list()

for t in regexp_tokenize(query, r'\b\w+\b'):
    #ngram 썼으니까 for문 한번더 돌리기
    for g in ngram(t): #ngram = > tokenizing.. 
        if g in V:
            result.append(list())
            j = w2i(g)
            pos = TDM[j]
            while pos != -1:
                i, freq, npos = Posting[pos]
                pos = npos

                result[-1].append(i)
            result[-1] = list(set(result[-1]))
                

In [21]:
print(result[0])

[256, 130, 259, 132, 133, 131, 10, 11, 140, 15, 278, 22, 280, 24, 155, 27, 157, 31, 33, 292, 37, 169, 41, 43, 45, 305, 49, 312, 315, 195, 76, 77, 335, 79, 81, 338, 212, 342, 86, 344, 216, 97, 232, 363, 236, 367, 112, 242, 243, 115, 373, 245, 119, 117, 249, 122, 252, 383]


In [23]:
print(sorted(result[0]))

[10, 11, 15, 22, 24, 27, 31, 33, 37, 41, 43, 45, 49, 76, 77, 79, 81, 86, 97, 112, 115, 117, 119, 122, 130, 131, 132, 133, 140, 155, 157, 169, 195, 212, 216, 232, 236, 242, 243, 245, 249, 252, 256, 259, 278, 280, 292, 305, 312, 315, 335, 338, 342, 344, 363, 367, 373, 383]


In [24]:
#구조체를 그대로 저장하기
from struct import pack, unpack
#pack 은 구조화된 애를 바이트 단위로 변환해준다.

In [25]:
import struct
help(struct)

Help on module struct:

NAME
    struct

MODULE REFERENCE
    https://docs.python.org/3.10/library/struct.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    Functions to convert between Python values and C structs.
    Python bytes objects are used to hold the data representing the C struct
    and also as format strings (explained below) to describe the layout of data
    in the C struct.
    
    The optional first format char indicates byte order, size and alignment:
      @: native order, size & alignment (default)
      =: native order, std. size & alignment
      <: little-endian, std. size & alignment
      >: big-endian, std. size & alignment
      !: same as >
    
    The remaining cha

In [38]:
pack('i',12)

b'\x0c\x00\x00\x00'

In [39]:
#다시 풀때는 언팩쓰면된다. 
unpack('iii' , pack('iii',1,1,-1))

(1, 1, -1)

# InvertedIndex 구조로 바꿔보기

In [40]:
#Posting 이 FILE 로 바뀌는거다. 
# 숫자로 나열되어있으면, 3개씩 끊어읽을거다. 1번문서가 2번나왔고 3번에 있다.123110123
# 그럼 결국 FILE 안에서 왔다갔다 해준다.

p1 = re.compile(r'\s+')

D = list()
V = list()
TDM  = dict()
#포스팅은 이제 FILE이다. 
# Posting = list()

#처음에 문서를 열거다. 포스팅이라는 구조로. 이번에는 인코딩 필요없다. 
Posting = open('posting.dat', 'wb')


for file in fileids('naver'):
    i = len(D)
    D.append(file)
    with open(file, 'r', encoding='utf8') as fp:
        for t in regexp_tokenize(p1.sub(' ', fp.read()), r'\b\w+\b'):
            for g in ngram(t):
                if g not in V:
                    V.append(g)
                j = V.index(g)

                if j not in TDM.keys():
                    #현재 위치를 기록. posting이 tell 로 말해줄거임. 
                    pos = Posting.tell()
                    Posting.write(pack('iii', i,1,-1))
                    TDM[j] = pos
                else:
                    pos = Posting.tell()
                    Posting.write(pack('iii', i,1,TDM[j]))
                    TDM[j] = pos

#포스팅 문닫아주기
Posting.close()

#### 저장한거까지 잘 되었다. 문제는 불러오기다.

In [41]:
#사용자에게서 쿼리를 받아보기
query = '예상보다 큰 격차의 참패로 여권이 대혼란에 빠지자 윤석열 대통령은 김행 여성가족부 장관 후보자 지명 철회 카드로 수습에 나섰다.'

result = list()

Posting = open('posting.dat', 'rb')

for t in regexp_tokenize(query, r'\b\w+\b'):
    #ngram 썼으니까 for문 한번더 돌리기
    for g in ngram(t): #ngram = > tokenizing.. 
        if g in V:
            result.append(list())
            
            j = w2i(g)
            pos = TDM[j]
            while pos != -1:
                Posting.seek(pos)
                #12바이트로  포스팅 읽은걸 언팩할거다. 
                i, freq, npos = unpack('iii', Posting.read(12))
                pos = npos

                result[-1].append(i)
            result[-1] = list(set(result[-1]))

Posting.close()

In [42]:
print(sorted(result[0]))

[10, 11, 15, 22, 24, 27, 31, 33, 37, 41, 43, 45, 49, 76, 77, 79, 81, 86, 97, 112, 115, 117, 119, 122, 130, 131, 132, 133, 140, 155, 157, 169, 195, 212, 216, 232, 236, 242, 243, 245, 249, 252, 256, 259, 278, 280, 292, 305, 312, 315, 335, 338, 342, 344, 363, 367, 373, 383]


## 빈도 고려한 코드 + 개별 파일별로 처리

In [43]:
#문서별로, 개별 코퍼스 별로 처리할거다. 

p1 = re.compile(r'\s+')

D = list()
V = list()
Dictionary  = dict()

#코퍼스 별로 생길거임. = 문서단위로 작업하고 나중에 합치겠다. 
#슬레이브 노드들한테 문서별로 분산해서 작업 시킬 수 있다. 그래서 map reduce 가 나오는거다. 
Posting = open('posting.dat', 'wb')


for file in fileids('naver'):
    i = len(D)
    D.append(file)
    with open(file, 'r', encoding='utf8') as fp:
        for t in regexp_tokenize(p1.sub(' ', fp.read()), r'\b\w+\b'):
            for g in ngram(t):
                if g not in V:
                    V.append(g)
                j = V.index(g)

                if j not in TDM.keys():
                    #현재 위치를 기록. posting이 tell 로 말해줄거임. 
                    pos = Posting.tell()
                    Posting.write(pack('iii', i,1,-1))
                    TDM[j] = pos
                else:
                    pos = Posting.tell()
                    Posting.write(pack('iii', i,1,TDM[j]))
                    TDM[j] = pos

#포스팅 문닫아주기
Posting.close()

In [44]:
fp = open('posting.dat', 'ab')
fp.tell()

3283656

In [45]:
fp.close()

In [47]:
fp = open('posting.dat', 'wb') #a-appending 마지막 위치
fp.tell()
#모드에 따라서 파일 위치가 다르기에 결과가 다르다.


0

In [48]:
fp.close()

### a-appending 마지막 위치 를 이용해서 코드 짜볼거임

In [51]:
#이번 코드에서는 위치도, 빈도도 필요가 없다. 

p1 = re.compile(r'\s+')

D = list()
V = list()
Dictionary  = dict()

# 이번에는 얘를 이용하지 않아볼거임 . 대신 훼이크로 빈 파일 하나 새로 만들었다가 바로 닫아놓을거임. 비어있는 파일 0으로 있다.
Posting = open('posting.dat', 'wb')
Posting.close()


#문서당 파일을 한번씩. 전체 코퍼스. 데이터셋.
for file in fileids('naver'):
    i = len(D) #글로벌에서 관리
    D.append(file) #전체문서집합 -> 메모리에 관리되어야할 애

    
    #코퍼스 1개 open 했을때
    with open(file, 'r', encoding='utf8') as fp:
        corpus = fp.read()

    
    #코퍼스를 하나 열때마다 기존 값 다 버리고 갱신하는거임. 
    localTDM = dict()
     #write 모드로 열어볼거임
    localPosting = open('local.dat', 'wb')

    
    #토크나이징 + 노멀라이징
    for t in regexp_tokenize(p1.sub(' ', corpus), r'\b\w+\b'):
        for g in ngram(t):
            #local 단위로 작업하기에 localTDM에서 확인할거임. #문서 1개 작업하는 중에, 단어가 처음 등장 
            if g not in localTDM.keys():
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1, -1)) #귀찮아서 빈도 1이라고 쓴거임.
                localTDM[g] = pos
            else:
                pos = localPosting.tell()
                localPosting.write(pack('ii', 1,localTDM[g]))
                localTDM[g] = pos
                
    localPosting.close() #로컬문서 여기서 긑--------------------------------------

    #Update; Local ->Global
    Posting = open('posting.dat', 'ab')
    localPosting = open('local.dat', 'rb') #파일을 로컬 읽기
    
    #localTDM 아이템즈로 열어볼거임 . k = 단어, v= 파일위치
    for k, v in localTDM.items():
        #k, v 찾아야해
        if k not in V:
            V.append(k) #없으면 V에다가 어펜드
        
        j = V.index(k) #j구하기. 몇번재 단어.

        freq = 0 #while문을 돌면서 1이상의 값이 기록될 거임. 
        pos = localPosting.tell()
        while pos != -1:
            localPosting.seek(pos)  
            f, npos = unpack('ii', localPosting.read(8)) #8바이트로 읽을거임-> unpack
            pos = npos #next 위치를 pos 에다가
            freq += 1 
                
        if j not in Dictionary.keys():
            pos = Posting.tell()    #freq를 알았기에 기록해야할것 : pos에다가 현재 위치 기록
            Posting.write(pack('iii', i, freq, -1))
            Dictionary[j] = pos   #딕셔너리가 처음이라면, 오픈해줘
        else:
            pos = Posting.tell()
            Posting.write(pack('iii', i, freq, Dictionary[j]))
            Dictionary[j]= 위치 # Dictionary[j]에다가 위치심기
        
    localPosting.close()
    Posting.close()

KeyboardInterrupt: 

In [ ]:
Posting.read(12)

In [ ]:
#로컬에서 작업 어떻게 되고 있는지 확인
# LocalTDM = {'단어': '로컬파일 위치', ...}
# LocalPosting
# 1다음위치1다음위치
#쓰다말았다. 그냥 수업들음. 나중에 필기 추가하기